In [47]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import utm
from rasterstats import zonal_stats
import rasterio
from rasterio.plot import show
import pandas as pd
import numpy as np
import gdal

In [2]:
points = gpd.read_file("/data/mangrove-data/Kathy/biomass_project/sites/lap_07-18_site1/lap_2018-07_site01_labels_m-nm/lap_2018-07_site01_labels_m-nm.shp")

In [3]:
xmin,ymin,xmax,ymax = points.total_bounds

In [4]:
length = 30
wide = 30

In [5]:
# La Paz- UTM Zone 12R
utm_min = utm.from_latlon(ymin, xmin)
utm_max = utm.from_latlon(ymax, xmax)

In [6]:
cols = list(range(int(np.floor(utm_min[0])), int(np.ceil(utm_max[0])), wide))
rows = list(range(int(np.floor(utm_min[1])), int(np.ceil(utm_max[1])), length))
rows.reverse()

In [7]:
polygons = []
for x in cols:
    for y in rows:
        polygons.append( Polygon([utm.to_latlon(x,y, utm_min[2], utm_min[3])[::-1], 
                                  utm.to_latlon(x+wide, y, utm_min[2], utm_min[3])[::-1], 
                                  utm.to_latlon(x+wide, y-length, utm_min[2], utm_min[3])[::-1], 
                                  utm.to_latlon(x, y-length, utm_min[2], utm_min[3])[::-1]]) )

grid = gpd.GeoDataFrame({'geometry':polygons})
grid.crs = {'init': 'epsg:4326'}
grid.to_file("/data/mangrove-data/Kathy/biomass_project/downsampling/testgrid.shp")

In [38]:
grid = gpd.read_file("/data/mangrove-data/Kathy/biomass_project/downsampling/testgrid.shp")
lap_1_2018_dem = "/data/mangrove-data/master/2018.07-mexico-lapaz/lap-2018-07-site1/dem/lap_2018-07_site1_120m_DEM_tc.tif"
gdf = grid.join(pd.DataFrame(zonal_stats(vectors=grid['geometry'], 
                                        raster=lap_1_2018_dem, 
                                        stats=['max','min', 'count', 'mean', 'std'],
                                        nodata=)), 
                how='left')


In [39]:
gdf

,FID,geometry,max,min,count,mean,std
0,0,"POLYGON ((-110.43808 24.17780, -110.43779 24.1...",NaN,NaN,0,NaN,NaN
1,1,"POLYGON ((-110.43808 24.17753, -110.43779 24.1...",NaN,NaN,0,NaN,NaN
2,2,"POLYGON ((-110.43809 24.17726, -110.43779 24.1...",1.338077,0.816875,7754,1.047493,0.044923
3,3,"POLYGON ((-110.43809 24.17698, -110.43779 24.1...",1.475163,0.832499,92666,1.173234,0.074899
4,4,"POLYGON ((-110.43809 24.17671, -110.43779 24.1...",1.646377,0.906725,198588,1.268766,0.053351
...,...,...,...,...,...,...,...
667,667,"POLYGON ((-110.43132 24.17154, -110.43102 24.1...",NaN,NaN,0,NaN,NaN
668,668,"POLYGON ((-110.43132 24.17127, -110.43102 24.1...",NaN,NaN,0,NaN,NaN
669,669,"POLYGON ((-110.43132 24.17100, -110.43102 24.1...",NaN,NaN,0,NaN,NaN
670,670,"POLYGON ((-110.43132 24.17073, -110.43103 24.1...",NaN,NaN,0,NaN,NaN


In [31]:
valid = gdf[gdf['count'] > 0]
print(np.max(valid['max']))
print(len(valid[valid['mean'] > 0]))

13.211426734924316
507


In [48]:
# get no data value
biomass = gdal.Open(dem_bio)
# convert gdal datasets to np arrays- including nodata pixels
np_biomass = np.array(biomass.GetRasterBand(1).ReadAsArray())
# first get rid of no data pixels- very large number
bad= np.amax(np_biomass)

In [49]:
dem_bio = f"/data/mangrove-data/Kathy/biomass_project/downsampling/lap-2018-07-site6/biomass/scaled_down_0.tif"
grid6 = "/data/mangrove-data/Kathy/biomass_project/downsampling/lap-2018-07-site6/grid.shp"
grid6 = gpd.read_file(grid6)
gdf = grid6.join(pd.DataFrame(zonal_stats(vectors=grid['geometry'], 
                                        raster=dem_bio, 
                                        stats=['max','min', 'count', 'mean', 'std'],
                                        nodata=bad)), 
                how='left')

In [52]:
gdf

,FID,geometry,max,min,count,mean,std
0,0,"POLYGON ((-110.31310 24.26178, -110.31281 24.2...",None,None,0,None,None
1,1,"POLYGON ((-110.31311 24.26151, -110.31281 24.2...",None,None,0,None,None
2,2,"POLYGON ((-110.31311 24.26124, -110.31281 24.2...",None,None,0,None,None
3,3,"POLYGON ((-110.31311 24.26097, -110.31281 24.2...",None,None,0,None,None
4,4,"POLYGON ((-110.31311 24.26070, -110.31281 24.2...",None,None,0,None,None
...,...,...,...,...,...,...,...
220,220,"POLYGON ((-110.30898 24.25906, -110.30869 24.2...",None,None,0,None,None
221,221,"POLYGON ((-110.30898 24.25879, -110.30869 24.2...",None,None,0,None,None
222,222,"POLYGON ((-110.30898 24.25851, -110.30869 24.2...",None,None,0,None,None
223,223,"POLYGON ((-110.30899 24.25824, -110.30869 24.2...",None,None,0,None,None
